<a href="https://colab.research.google.com/github/LakshX413/Deep-Learning-FastAI/blob/main/LLM_Chat_with_data_trial_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#connect to supbase - https://supabase.com/docs/guides/ai/google-colab
#python - https://supabase.com/docs/reference/python/initializing


In [1]:
!pip install supabase
!pip install streamlit
!pip install psycopg2
!pip install openai==0.28
# !pip install re

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.9/169.9 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.70.0
    Uninstalling openai-1.70.0:
      Successfully uninstalled openai-1.70.0


In [2]:
from google.colab import userdata
import os

os.environ['openai_key'] = userdata.get('OPENAI_API_KEY')
os.environ['supabase_pwd'] = userdata.get('SUPBASE_PWD')
os.environ['NGROK_TOKEN'] = userdata.get('NGROK_TOKEN')

In [9]:
%%writefile doc_chat.py
# code = '''

import streamlit as st
import psycopg2
import psycopg2.extras  # Import extras for DictCursor
import openai
import re
import os

openai.api_key = os.environ['openai_key']


# Supabase connection details
db_config = {
    "host": "aws-0-ap-south-1.pooler.supabase.com",
    "user": "postgres.onqljptkxdzpzowyjqkk",
    "password": os.environ['supabase_pwd'],
    "database": "postgres",
    "port": "6543",
    "table_name": "final_sales_reduced_wkstart"
}

def get_table_columns(db_config):
    """Retrieves column names and types from the Supabase table."""
    try:
        # Actually use the conn_params you've created
        conn_params = {
            "host": db_config["host"],
            "user": db_config["user"],
            "password": db_config["password"],
            "database": db_config["database"],
            "port": db_config["port"],
        }
        conn = psycopg2.connect(**conn_params)  # Use conn_params here
        cursor = conn.cursor()

        # Ensure table name is lowercase
        query = """
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_name = %s;
        """
        cursor.execute(query, (db_config['table_name'].lower(),))  # Convert to lowercase
        columns = cursor.fetchall()
        conn.close()

        if not columns:
            st.error(f"No columns found for table '{db_config['table_name']}'. Check if the table exists.")
            return []

        column_info = [{"name": col[0], "type": col[1], "description": ""} for col in columns]
        # st.write("Debug: Retrieved Columns ->", column_info)  # Debugging output
        return column_info
    except Exception as e:
        st.error(f"Error retrieving table columns: {e}")
        return []

def execute_sql(sql, db_config):
    """Executes a SQL query and returns the results."""
    try:
        conn_params = {key: db_config[key] for key in ["host", "user", "password", "database", "port"]}
        conn = psycopg2.connect(**conn_params)  # Use the connection parameters
        cursor = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)  # Use DictCursor for better output
        cursor.execute(sql)
        results = cursor.fetchall()
        conn.close()
        return results
    except Exception as e:
        st.error(f"SQL execution error: {e}")
        return {"error": str(e)}

def generate_sql(prompt, openai_key):
    """Generates SQL using OpenAI."""
    openai.api_key = os.environ['openai_key']
    try:
        response = openai.Completion.create(
            engine="gpt-3.5-turbo-instruct",
            prompt=prompt,
            max_tokens=250,
        )
        sql = re.search(r"SELECT.*?;", response.choices[0].text, re.DOTALL)
        if sql:
            return sql.group(0)
        else:
            return None
    except Exception as e:
        st.error(f"OpenAI API error: {e}")
        return None

def create_schema_prompt(table_name, columns):
    """Creates a prompt explaining the table schema."""
    if not columns:
        return None

    column_descriptions = ", ".join(
        f"{col['name']} ({col['type']}): {col['description']}" for col in columns
    )

    prompt = f"""
    You are an AI that translates user questions into SQL queries for a PostgreSQL database.
    The database table name is: {table_name}.
    The table columns are: {column_descriptions}.

    Example:
    User Question: What are the names of people older than 30?
    SQL Query: SELECT "Name" FROM {table_name} WHERE "Age" > 30;

    User Question:
    """
    return prompt

# Streamlit UI
st.title("Recon Sales Data Chatbot")
user_input = st.text_input("Ask a question about the data:")

columns = get_table_columns(db_config)  # Get column names.
if columns:
    schema_prompt = create_schema_prompt(db_config['table_name'], columns)

    if schema_prompt and user_input:
        # prompt = schema_prompt + user_input + "\nSQL Query:"
        prompt = f"{schema_prompt}{user_input}\nSQL Query:"
        sql = generate_sql(prompt, os.environ['openai_key'])
        if sql:
            results = execute_sql(sql, db_config)
            st.write("SQL Query:", sql)
            st.write("Results:", results)
            response = openai.Completion.create(
                engine="gpt-3.5-turbo-instruct",
                prompt=prompt + sql + "\n Text Response:",
                max_tokens=250,
                api_key=os.environ['openai_key']  # Add the API key here
            )
            st.write("Text Response:", response.choices[0].text)
        else:
            st.write("Could not generate a valid SQL query.")
else:
    st.write("Could not retrieve table column information. Please check database connection.")

Overwriting doc_chat.py


In [8]:
##unsure why but this code needs to re-run twice
!pip install pyngrok
from pyngrok import ngrok

# Set your ngrok authtoken
ngrok.set_auth_token(os.environ['NGROK_TOKEN'])  # Replace with your actual authtoken

!streamlit run doc_chat.py &>/dev/null&
ngrok_tunnel = ngrok.connect(8501)
print(ngrok_tunnel.public_url)

https://fce8-34-138-122-180.ngrok-free.app


In [ ]:
# import urllib
# print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

In [ ]:
# !streamlit run doc_chat.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501